# Membership Inference Competition (MICO) @ IEEE SatML 2023: Purchase-100

Welcome to the MICO competition!

This notebook will walk you through the process of creating and packaging a submission to one of the challenges.

Let's start by downloading and extracting the archive for the Purchase-100 challenge.

**NOTE**: Public anonymous access to the competition data is disabled. 
Upon registering for the competition, you will be shown a URL with an embedded bearer token that you must use instead of the URL below.

In [1]:
import os
import urllib

from torchvision.datasets.utils import download_and_extract_archive
from sklearn.metrics import roc_curve, roc_auc_score

from mico_competition.scoring import tpr_at_fpr, score, generate_roc, generate_table
from sklearn.metrics import roc_curve, roc_auc_score

import numpy as np
import torch
import csv

from torch.autograd import Variable
from sklearn import metrics
from tqdm.notebook import tqdm
from torch.distributions import normal
from torch.utils.data import DataLoader, Dataset
from mico_competition import ChallengeDataset, load_purchase100, load_model

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import torch as ch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset
from opacus.utils.batch_memory_manager import BatchMemoryManager

from mico_competition import MLP

In [2]:
def accuracy(preds: ch.Tensor, labels: ch.Tensor) -> float:
    return (preds == labels).mean()

In [3]:
def train(model: nn.Module,
          train_loader: DataLoader,
          criterion,
          optimizer: optim.Optimizer,
          batch_size: int):
    model.train()

    losses = []
    top1_acc = []

    disable_dp = True
    max_physical_batch_size = 128
    with BatchMemoryManager(
        data_loader=train_loader,
        max_physical_batch_size=max_physical_batch_size,
        optimizer=optimizer
    ) as memory_safe_data_loader:

        if disable_dp:
            data_loader = train_loader
        else:
            data_loader = memory_safe_data_loader

        # BatchSplittingSampler.__len__() approximates (badly) the length in physical batches
        # See https://github.com/pytorch/opacus/issues/516
        # We instead heuristically keep track of logical batches processed
        logical_batch_len = 0
        for i, (inputs, target) in enumerate(data_loader):
            inputs, target = inputs.cuda(), target.cuda()

            logical_batch_len += len(target)
            if logical_batch_len >= batch_size:
                logical_batch_len = logical_batch_len % max_physical_batch_size

            optimizer.zero_grad()
            output = model(inputs)
            loss = criterion(output, target)

            preds = np.argmax(output.detach().cpu().numpy(), axis=1)
            labels = target.detach().cpu().numpy()
            acc = accuracy(preds, labels)

            losses.append(loss.item())
            top1_acc.append(acc)

            loss.backward()
            optimizer.step()
    return model

In [4]:
def train_model(loader):
    batch_size = 512
    learning_rate = 0.001
    lr_scheduler_step = 5
    lr_scheduler_gamma = 0.9
    num_epochs = 30

    model = MLP()
    model.cuda()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    sample_rate = 1 / len(loader)
    num_steps = int(len(loader) * num_epochs)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=lr_scheduler_step, gamma=lr_scheduler_gamma)

    for _ in range(num_epochs):
        train(model, loader, criterion, optimizer, batch_size)
        scheduler.step()
    
    return model

In [5]:
def normalize_preds(preds):
    # Normalize to unit interval
    min_prediction = np.min(preds)
    max_prediction = np.max(preds)
    preds = (preds - min_prediction) / (max_prediction - min_prediction)
    return preds

In [6]:
def get_losses(model, features, labels):
    output = model(features)
    criterion = torch.nn.CrossEntropyLoss(reduction='none')
    predictions = -criterion(output, labels).detach().cpu().numpy()
    return predictions

In [7]:
def get_gradient_norm(model, features, labels):
    criterion = torch.nn.CrossEntropyLoss(reduction='none')
    features_collected = []
    focus = 0
    for feature, label in zip(features, labels):
        model.zero_grad()
        feature_var = Variable(feature)
        output = model(feature)
        loss = criterion(torch.unsqueeze(output, 0), torch.unsqueeze(label, 0))
        loss.backward()
        features_collected.append([torch.linalg.norm(x.grad.detach().cpu()).item() for x in model.parameters()])
    features_collected = np.array(features_collected)[:, focus]
    return features_collected.reshape(-1, 1)

In [8]:
def ascent_recovery(model, features, labels):
    criterion = torch.nn.CrossEntropyLoss(reduction='none')
    n_times = 10
    step_size = 0.001
    final_losses, final_dist = [], []
    for i, (feature, label) in enumerate(zip(features, labels)):
        model.zero_grad()
        feature_var = Variable(feature.clone().detach(), requires_grad=True)
        for j in range(n_times):
            feature_var = Variable(feature_var.clone().detach(), requires_grad=True)
            loss = criterion(torch.unsqueeze(model(feature_var), 0), torch.unsqueeze(label, 0))
            loss.backward(ch.ones_like(loss), retain_graph=True)
            with ch.no_grad():
                feature_var.data -= step_size * feature_var.data
                loss_new = criterion(torch.unsqueeze(model(feature_var), 0), torch.unsqueeze(label, 0))
        # Get reduction in loss
        final_losses.append(loss.item() - loss_new.item())
        # Get change in data (norm)
        final_dist.append(ch.norm(feature_var.data - feature.data).detach().cpu().numpy())
    final_losses = np.array(final_losses)
    final_dist = np.array(final_dist)
    return np.stack((final_losses, final_dist), 1)

In [9]:
def gradient_and_robustness_new(model, features, labels):
    features_1 = ascent_recovery(model, features, labels)
    features_2 = get_gradient_norm(model, features, labels)
    features_3 = get_losses(model, features, labels).reshape(-1, 1)
    combined_feratures = np.concatenate((features_1, features_2, features_3), 1)
    return combined_feratures

In [10]:
def collect_from_batch(loader):
    X, Y = [], []
    for batch in loader:
        X.append(batch[0])
        Y.append(batch[1])
    X = ch.cat(X, 0)
    Y = ch.cat(Y, 0)
    return X, Y

In [14]:
def n_datasets_generator(data, member, n_times, n_sample):
    batch_size = 512
    n_workers = 1
    for _ in range(n_times):
        indices = np.random.choice(len(data[0]), n_sample, replace=False)
        dataset_without = TensorDataset(data[0][indices], data[1][indices])
        X_wanted = ch.cat((data[0][indices], member[0].view(1, -1)))
        Y_wanted = ch.cat((data[1][indices], member[1].view(1)))
        dataset = TensorDataset(X_wanted, Y_wanted)
        # Make loaders
        train_loader = DataLoader(
            dataset,
            batch_size=batch_size,
            num_workers=n_workers,
            pin_memory=True,
        )
        train_loader_without = DataLoader(
            dataset_without,
            batch_size=batch_size,
            num_workers=n_workers,
            pin_memory=True,
        )
        yield train_loader, train_loader_without

In [15]:
# Collect "training data" using models from train split
def collect_training_data():
    CHALLENGE = "purchase100"
    LEN_TRAINING = 150000
    LEN_CHALLENGE = 100

    scenarios = os.listdir(CHALLENGE)

    dataset = load_purchase100(dataset_dir="/u/as9rw/work/MICO/data")

    collected_features = {x:[] for x in scenarios}
    collected_labels = {x:[] for x in scenarios}
    phase = "train"
    for scenario in tqdm(scenarios, desc="scenario"):
        root = os.path.join(CHALLENGE, scenario, phase)
        for model_folder in tqdm(sorted(os.listdir(root), key=lambda d: int(d.split('_')[1])), desc="model"):
            path = os.path.join(root, model_folder)
            challenge_dataset = ChallengeDataset.from_path(path, dataset=dataset, len_training=LEN_TRAINING)
            # Get 'rest' data (for shadow-model training)
            challenge_points = challenge_dataset.get_rest()
            
            # Create generator to re-sample data from given data
            # Which will then be used for training shadow models
            challenge_dataloader = torch.utils.data.DataLoader(challenge_points, batch_size=2*LEN_CHALLENGE)
            features, labels = collect_from_batch(challenge_dataloader)
            features, labels = features.cuda(), labels.cuda()
            
            for feature, label in zip(features, labels):
                n_times = 3 #10
                n_sample = 1000 #100000
                generator = n_datasets_generator((features, labels), (features[0], labels[0]), n_times, n_sample - 1)

                X_, Y_ = [], []
                for (loader, loader_without) in tqdm(generator, desc="Training shadow models", total=n_times):
                    # Train shadow models
                    shadow = train_model(loader)
                    shadow_without = train_model(loader_without)
                
                    feature_with = gradient_and_robustness_new(shadow, feature.view(1, -1), label.view(1))
                    feature_without = gradient_and_robustness_new(shadow_without, feature.view(1, -1), label.view(1))
                    X_.append(feature_with)
                    X_.append(feature_without)
                    Y_.append(0)
                    Y_.append(1)
                return X_, Y_
            
            # Load actual victim model
            model = load_model('purchase100', path)
            model.cuda()
            
            # Collect features
            collected_features[scenario].append(processed_features)
            
            # Get labels for membership
            collected_labels[scenario].append(challenge_dataset.get_solutions())
            np_y = np.array(challenge_dataset.get_solutions())
    
    for sc in scenarios:
        collected_features[sc] = np.concatenate(collected_features[sc], 0)
        collected_labels[sc] = np.concatenate(collected_labels[sc], 0)

    return collected_features, collected_labels

In [16]:
X_for_meta, Y_for_meta = collect_training_data()

Successfully loaded the Purchase-100 dataset consisting of 197324 records and 600 attributes.


scenario:   0%|          | 0/3 [00:00<?, ?it/s]

model:   0%|          | 0/100 [00:00<?, ?it/s]

Training shadow models:   0%|          | 0/3 [00:00<?, ?it/s]

RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/u/as9rw/anaconda3/envs/mico/lib/python3.8/site-packages/torch/utils/data/_utils/worker.py", line 202, in _worker_loop
    data = fetcher.fetch(index)
  File "/u/as9rw/anaconda3/envs/mico/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 44, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/u/as9rw/anaconda3/envs/mico/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 44, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/u/as9rw/anaconda3/envs/mico/lib/python3.8/site-packages/torch/utils/data/dataset.py", line 171, in __getitem__
    return tuple(tensor[index] for tensor in self.tensors)
  File "/u/as9rw/anaconda3/envs/mico/lib/python3.8/site-packages/torch/utils/data/dataset.py", line 171, in <genexpr>
    return tuple(tensor[index] for tensor in self.tensors)
Runt

In [ ]:
CHALLENGE = "purchase100"
scenarios = os.listdir(CHALLENGE)

bins = 100
for sc in scenarios:
    plt.hist(X_for_meta[sc][Y_for_meta[sc] == 0], bins, alpha=0.5, label='non-members')
    plt.hist(X_for_meta[sc][Y_for_meta[sc] == 1], bins, alpha=0.5, label='members')
    plt.legend(loc='upper right')
    plt.show()

In [ ]:
CHALLENGE = "purchase100"
scenarios = os.listdir(CHALLENGE)

bins = 100
for sc in scenarios:
    wanted_zero, wanted_one = X_for_meta[sc][Y_for_meta[sc] == 0], X_for_meta[sc][Y_for_meta[sc] == 1]
    index = 3
    # Layers:
    # 4 : 0.57
    # 3 : 0.579
    # 2 : 0.5733
    # 1 : 0.583
    plt.scatter(wanted_zero[:, index], wanted_zero[:, 0], label='non-members', s=4, alpha=0.5)
    threshold = 10
    print((np.mean(wanted_zero[:, index] >= threshold) + np.mean(wanted_one[:, index] <= threshold))/2)
    plt.scatter(wanted_one[:, index], wanted_one[:, 0], label='members', s=4, alpha=0.5)
    plt.legend(loc='upper right')
    plt.show()

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn import preprocessing
from sklearn.inspection import permutation_importance
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

# Train different meta-classifiers per scenario
CHALLENGE = "purchase100"
scenarios = os.listdir(CHALLENGE)
meta_clfs = {x: RandomForestClassifier(max_depth=3) for x in X_for_meta.keys()}
# meta_clfs = {x: LogisticRegression(n_jobs=-1) for x in X_for_meta.keys()}
# meta_clfs = {x: GaussianNB() for x in X_for_meta.keys()}

for sc in scenarios:
    X_train, X_test, y_train, y_test = train_test_split(X_for_meta[sc], Y_for_meta[sc], test_size=0.5)
    # X_train, X_test = X_train[:, desired], X_test[:, desired]
    
    meta_clfs[sc].fit(X_train, y_train)
    preds = meta_clfs[sc].predict_proba(X_test)[:, 1]
    preds_train = meta_clfs[sc].predict_proba(X_train)[:, 1]
    
    # Round predictions?
    preds = np.round(preds, 4)
    preds_train = np.round(preds_train, 4)
    
    print(f"{sc} AUC (train): {roc_auc_score(y_train, preds_train)}")
    scores = score(y_test, preds)
    scores.pop('fpr', None)
    scores.pop('tpr', None)
    display(pd.DataFrame([scores]))
    
#     scores = score(y_test, preds)
#     fpr = scores['fpr']
#     tpr = scores['tpr']
    #fig = generate_roc(fpr, tpr)
    #fig.suptitle(f"{sc}", x=-0.1, y=0.5)
    #fig.tight_layout(pad=1.0)
    
#     result = permutation_importance(meta_clfs[sc], X_test, y_test, n_repeats=10, random_state=42, n_jobs=-1)
#     forest_importances = pd.Series(result.importances_mean) #, index=feature_names)

#     fig, ax = plt.subplots()
#     forest_importances.plot.bar(yerr=result.importances_std, ax=ax)
#     ax.set_title(f"{sc} : Feature importances using permutation")
#     ax.set_ylabel("Mean accuracy decrease")
#     fig.tight_layout()
#     plt.show()

###### Use given datapoint, train models w and w/o that point
# Adapt KL test (from our SaTML paper) to make prediction

In [ ]:
# Another idea- perform one step of GD on model with datapoint, and compare gradient updates with
# cases where member was not seen before, and use this as a feature for a meta-classifier

In [ ]:
# Plain old permutation-invariant network-based meta-classifier, but also take as input
# The raw datapoint. Hope meta-classifier learns to form associations, but not sure how to
# design such a meta-classifier (modifications). 

In [ ]:
def pick_top_k_points(model, data, discard_percentage):
    criterion = torch.nn.CrossEntropyLoss(reduction='none')
    dataloader = torch.utils.data.DataLoader(data, batch_size=1000, shuffle=False)
    loss_vals = []
    for x, y in dataloader:
        loss_vals.append(criterion(model(x), y).detach())
    loss_vals = ch.cat(loss_vals).cpu().numpy()

In [ ]:
def approximate_retraining_attack(model, x, challenge_dataset):
    LEN_TRAINING = 150000
    LEN_CHALLENGE = 100
    rest_points = challenge_dataset.get_rest()
    challenge_dataloader = torch.utils.data.DataLoader(rest_points, batch_size=LEN_TRAINING - (2*LEN_CHALLENGE))
    rest_x, rest_y = next(iter(challenge_dataloader))
    # Get loss values for these 

In [ ]:
CHALLENGE = "purchase100"
LEN_TRAINING = 150000
LEN_CHALLENGE = 100

scenarios = os.listdir(CHALLENGE)
phases = ['dev', 'final', 'train']

dataset = load_purchase100(dataset_dir="/u/as9rw/work/MICO/data")

for scenario in tqdm(scenarios, desc="scenario"):
    for phase in tqdm(phases, desc="phase"):
        root = os.path.join(CHALLENGE, scenario, phase)
        for model_folder in tqdm(sorted(os.listdir(root), key=lambda d: int(d.split('_')[1])), desc="model"):
            path = os.path.join(root, model_folder)
            challenge_dataset = ChallengeDataset.from_path(path, dataset=dataset, len_training=LEN_TRAINING)
            challenge_points = challenge_dataset.get_challenges()
            
            model = load_model('purchase100', path)
            challenge_dataloader = torch.utils.data.DataLoader(challenge_points, batch_size=2*LEN_CHALLENGE)
            features, labels = next(iter(challenge_dataloader))
            
            # Weighted neighborhood loss fluctuation
            # Got score
            #predictions = neighborhood_robustness(model, features, labels, 0.2, 20)
            #predictions = normalize_preds(predictions)
            #predictions = 1 - predictions

            # This is where you plug in your membership inference attack
            # Combine preds from both
            # Got 0.1106 score
            # predictions = neighborhood_and_loss(model, features, 0.01, 10)
            # predictions = normalize_preds(predictions)
            
            # Meta-classifier :Random Forest, directly across all data/models
            # Got 0.1121 score
            # Scenario-wise meta-clfs got 0.1231 score
            # processed_features = neighborhood_and_loss(model, features, labels, as_features=True)
            
            # Meta-classifier: Random forest, on gradient updates
            # Got 0.0716 score
            # processed_features = get_gradient_norm(model, features, labels)
            
            # Meta-classifier: Random forest, on gradient updates, loss, and robustness
            # Got 0.1224 score
            # processed_features = gradient_and_robustness(model, features, labels)
            
            # Meta-classifier on multiple gradient updates (RF)
            # Got 0.1265 score
            # processed_features = get_gradient_update_norms(model, features, labels)
            
            # Meta-classifier on multiple gradient updates, neighborhood loss, and loss
            # Got score
            # processed_features = get_neighborhood_and_loss(model, features, labels)
            
            # Meta-classifier on gradient norm (0th), ascent loss diff, and straight-up loss
            # Got 0.1342 score
            processed_features = gradient_and_robustness_new(model, features, labels)
            
            predictions = meta_clfs[scenario].predict_proba(processed_features)[:, 1]

            assert np.all((0 <= predictions) & (predictions <= 1))

            with open(os.path.join(path, "prediction.csv"), "w") as f:
                 csv.writer(f).writerow(predictions)

## Scoring

Let's see how the attack does on `train`, for which we have the ground truth. 
When preparing a submission, you can use part of `train` to develop an attack and a held-out part to evaluate your attack. 

In [ ]:
FPR_THRESHOLD = 0.1

all_scores = {}
phases = ['train']

for scenario in tqdm(scenarios, desc="scenario"): 
    all_scores[scenario] = {}    
    for phase in tqdm(phases, desc="phase"):
        predictions = []
        solutions  = []

        root = os.path.join(CHALLENGE, scenario, phase)
        for model_folder in tqdm(sorted(os.listdir(root), key=lambda d: int(d.split('_')[1])), desc="model"):
            path = os.path.join(root, model_folder)
            predictions.append(np.loadtxt(os.path.join(path, "prediction.csv"), delimiter=","))
            solutions.append(np.loadtxt(os.path.join(path, "solution.csv"),   delimiter=","))

        predictions = np.concatenate(predictions)
        solutions = np.concatenate(solutions)
        
        scores = score(solutions, predictions)
        all_scores[scenario][phase] = scores

Let's plot the ROC curve for the attack and see how the attack performed on different metrics

In [ ]:
import matplotlib.pyplot as plt
import matplotlib

for scenario in scenarios:
    fpr = all_scores[scenario]['train']['fpr']
    tpr = all_scores[scenario]['train']['tpr']
    fig = generate_roc(fpr, tpr)
    fig.suptitle(f"{scenario}", x=-0.1, y=0.5)
    fig.tight_layout(pad=1.0)

In [ ]:
import pandas as pd

for scenario in scenarios:
    print(scenario)
    scores = all_scores[scenario]['train']
    scores.pop('fpr', None)
    scores.pop('tpr', None)
    display(pd.DataFrame([scores]))

## Packaging the submission

Now we can store the predictions into a zip file, which you can submit to CodaLab.

In [ ]:
import zipfile

phases = ['dev', 'final']
experiment_name = "ascent_loss_gradnorm"

with zipfile.ZipFile(f"{experiment_name}.zip", 'w') as zipf:
    for scenario in tqdm(scenarios, desc="scenario"): 
        for phase in tqdm(phases, desc="phase"):
            root = os.path.join(CHALLENGE, scenario, phase)
            for model_folder in tqdm(sorted(os.listdir(root), key=lambda d: int(d.split('_')[1])), desc="model"):
                path = os.path.join(root, model_folder)
                file = os.path.join(path, "prediction.csv")
                if os.path.exists(file):
                    zipf.write(file)
                else:
                    raise FileNotFoundError(f"`prediction.csv` not found in {path}. You need to provide predictions for all challenges")